## ASSIGNMENT-2

Roll Numbers: 2023121013, 2023122002, 2023121006

Names: Himanshu Singh, Srinath Bhamidipati, Pavan Karke

### Instructions
 * Fill in the roll numbers and names of all the teams members in the cell above.
 * Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
 * All the code and result files should be uploaded in the github classroom.
 * You can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
 * All the representations are expected to be in a right-hand coordinate system. All the functions related to transformation matrices, quaternions, and 3D projection are expected to be coded by you.
 * You could split the Jupyter Notebook cells where TODO is written, but please try to avoid splitting/changing the structure of other cells.
 * All the visualization should be done inside the notebook unless specified otherwise.
 * Plagiarism will lead to heavy penalty.

## G2O Motion Model 

Using the following motion model, you have to first generate the "initialization" for all the poses/vertices using the "Given" information. Just like in the 1D case.
$$x_{k+1} = x_{k} + \Delta x_{(k,k+1)} \cos(\theta_k) - \Delta y_{(k,k+1)} \sin(\theta_k) \\
y_{k+1} = y_{k} + \Delta y_{(k,k+1)} \cos(\theta_k) + \Delta x_{(k,k+1)} \sin(\theta_k) \\
\theta_{k+1} = \theta_{k}+  \Delta \theta_{(k,k+1)}$$

Even the loop closure nodes are related by the above model, except that it need not necessarily be consecutive nodes k and k+1.

## PART 2

### 1. ICP Alignment with LieTorch (5 marks)
Resources: https://github.com/princeton-vl/lietorch/tree/master

Perform ICP Alignment of 2 PointClouds using LieTorch. Similar to what was expected in Part 1, you will be given a random Transformation matrix from the getTransform function. You are expected to transform the Toothless Point Cloud by this transformation, add noise to both sets of points (original and transformed) and estimate both, the Transformation matrix and the original point cloud.

Visualize your results using open3d

In [1]:
# Imports

import os

import lietorch
import numpy as np
import open3d as o3d
import torch

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def add_gaussian_noise(X, alpha=0.01):
    """ Adds Gaussian noise to a set of homogeneous points. """

    return X + torch.hstack([alpha * torch.randn(X.shape[0], 3), torch.zeros((X.shape[0], 1))])

def apply_transformation(X, T):
    """ Applies a 4x4 transformation matrix to a set of homogeneous points. """

    return X @ T.T

def euclidean_coordinates(X):
    """ Converts a set of points in homogeneous coordinate system to Euclidean coordinate system. """

    return X[:, :3] / X[:, [3]]

def exponential_map(xi_translation, xi_rotation):
    """ Converts a lie algebra entity to transformation matrix. """

    return lietorch.SE3.exp(torch.hstack([xi_translation, xi_rotation])).matrix()

def homogeneous_coordinates(X):
    """ Converts a set of points in Euclidean coordinate system to homogeneous coordinate system. """

    return torch.hstack([X, torch.ones((X.shape[0], 1))])

def inverse_transformation(T):
    """ Computes the inverse transformation of a 4x4 matrix. """

    rotation = T[:3, :3]
    translation = T[:3, [3]]
    return torch.vstack([torch.hstack([rotation.T, - rotation.T @ translation]), torch.Tensor([0, 0, 0, 1])])

def mean_squared_error(X_1, X_2):
    """ Computes the mean squared error between two point clouds. """

    return torch.mean(torch.sum((X_1 - X_2) ** 2, dim=1))

In [3]:
def getTransform():
    """ Generates a random 4x4 transformation matrix. """

    R = o3d.geometry.get_rotation_matrix_from_xyz(np.random.rand(3) * 2 * np.pi)
    T = np.random.rand(3) * 100
    transformation = np.vstack((np.hstack((R, T.reshape(-1, 1))), [0, 0, 0, 1]))
    return torch.Tensor(transformation)

In [4]:
def icp_alignment(
    T, max_iterations=750, tolerance=1e-2, lr_translation=1e-1, lr_rotation=1e-5, lr_point_cloud=1e-2
):
    """ Given a transformation matrix, transforms toothless.ply
    and recovers the transformation using Lie Group Optimization."""

    # Read the point cloud
    pcd = o3d.io.read_point_cloud("data/toothless.ply")
    X_P1 = torch.Tensor(np.asarray(pcd.points))

    # Homogeneous coordinate system
    X_P1 = homogeneous_coordinates(X_P1)

    # Apply the transformation
    X_P2 = apply_transformation(X_P1, T)

    # Add Gaussian noise to simulate practical scenarios
    X_P1 = add_gaussian_noise(X_P1)
    X_P2 = add_gaussian_noise(X_P2)

    # Predicted transformation and point cloud
    xi_translation = torch.zeros(3, requires_grad=True)
    xi_rotation = torch.zeros(3, requires_grad=True)
    X_P1.requires_grad_()

    # Setup optimizer for the lie group object
    optimizer = torch.optim.SGD([
        { 'params': xi_translation, 'lr': lr_translation },
        { 'params': xi_rotation, 'lr': lr_rotation },
        { 'params': X_P1, 'lr': lr_point_cloud }
    ], momentum=0.1)

    # Iterate until termination condition
    for iteration in range(max_iterations):

        # Apply the current transformation
        X_P2_pred = apply_transformation(X_P1, exponential_map(xi_translation, xi_rotation))

        # Compute the loss
        loss = mean_squared_error(X_P2, X_P2_pred)
        print(loss.item())

        # Optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Check for convergence
        if loss < tolerance:
            break

    # Stop computing the gradients for the predicted transform
    xi_translation = xi_translation.detach()
    xi_rotation = xi_rotation.detach()
    X_P1 = X_P1.detach()

    # Apply the predicted transformation
    T_pred = exponential_map(xi_translation, xi_rotation)
    X_P2_aligned = apply_transformation(X_P2, inverse_transformation(T_pred))

    # Compute RMSE between the two transformed point clouds
    error = torch.sqrt(mean_squared_error(X_P1, X_P2_aligned))

    # Point clouds
    X = [ euclidean_coordinates(X_P1), euclidean_coordinates(X_P2_aligned) ]

    return T_pred, error, X

T = getTransform()
T_pred, alignment_error, X = icp_alignment(T)
T_error = torch.sqrt(torch.mean((T_pred - T) ** 2))

print('Original Transformation:\n', T)
print('Predicted Transformation:\n', T_pred)
print('Alignment Error:', alignment_error)
print('Error in Transformation:', T_error)

# Visualization
vis = o3d.visualization.Visualizer()
vis.create_window()
PC_1 = o3d.geometry.PointCloud()
PC_2 = o3d.geometry.PointCloud()
PC_1.points = o3d.utility.Vector3dVector(X[0].numpy())
PC_2.points = o3d.utility.Vector3dVector(X[1].numpy())
vis.add_geometry(PC_1)
vis.add_geometry(PC_2)
vis.poll_events()
vis.update_renderer()
vis.run()
vis.destroy_window()

31077.068359375
23789.693359375
18761.486328125
15813.4736328125
14125.5712890625
13151.2626953125
12573.6220703125
12214.8701171875
11976.42578125
11803.8720703125
11667.197265625
11549.8212890625
11442.548828125
11340.265625
11240.111328125
11140.478515625
11040.47265625
10939.578125
10837.5283203125
10734.171875
10629.4296875
10523.2841796875
10415.73046875
10306.791015625
10196.4970703125
10084.880859375
9971.994140625
9857.88671875
9742.6025390625
9626.1962890625
9508.720703125
9390.2275390625
9270.7744140625
9150.4169921875
9029.2080078125
8907.1962890625
8784.4462890625
8661.00390625
8536.92578125
8412.2607421875
8287.05859375
8161.37353515625
8035.2548828125
7908.7568359375
7781.92822265625
7654.82470703125
7527.49462890625
7399.994140625
7272.37744140625
7144.70361328125
7017.033203125
6889.42626953125
6761.94189453125
6634.6484375
6507.609375
6380.89404296875
6254.576171875
6128.724609375
6003.41943359375
5878.72900390625
5754.7353515625
5631.51513671875
5509.14599609375
5387

### 2. ICP-SLAM with LieTorch (10 marks)

For this part, you will be given a set of `N` transformation matrices (Each relative to position 0). Add a random amount of noise to each of the `N` sets of points. Estimate the Transforms as well as the original point cloud using LieTorch.

Visualize your results using open3d

In [5]:
def getTransforms(n):
    # Output shape - (n, 4, 4)

    T = []
    for _ in range(n):
        T.append(getTransform())

    return torch.stack(T)

In [6]:
def icp_slam(
    T, max_iterations=750, tolerance=1e-2, lr_translation=1e-1, lr_rotation=1e-5, lr_point_cloud=1e-4
):
    """ Given set of transformation matrices, transforms toothless.ply
    and recovers the transformation using Lie Group Optimization."""

    # Read the point cloud
    pcd = o3d.io.read_point_cloud("data/toothless.ply")
    X = torch.Tensor(np.asarray(pcd.points))

    # Dimensions
    num_points = X.shape[0]
    num_frames = T.shape[0]

    # Homogeneous coordinate system
    X = homogeneous_coordinates(X)

    # Apply the transformation
    X_frames = []
    for idx in range(num_frames):
        X_frames.append(apply_transformation(X, T[idx]))

    # Add Gaussian noise to simulate practical scenarios
    X = add_gaussian_noise(X, alpha=0)
    for idx in range(num_frames):
        X_frames[idx] += add_gaussian_noise(X_frames[idx], alpha=0)

    # Predict the transformation
    xi_translation = torch.zeros((num_points, 3), requires_grad=True)
    xi_rotation = torch.zeros((num_points, 3), requires_grad=True)
    X.requires_grad_()

    # Setup optimizer for the predicted transformation
    optimizer = torch.optim.SGD([
        { 'params': xi_translation, 'lr': lr_translation },
        { 'params': xi_rotation, 'lr': lr_rotation },
        { 'params': X, 'lr': lr_point_cloud }
    ], momentum=0.1)

    # Iterate until termination condition
    for iteration in range(max_iterations):

        # Apply the current transformation
        X_frames_pred = []
        for idx in range(num_frames):
            X_frames_pred.append(apply_transformation(X, exponential_map(xi_translation[idx], xi_rotation[idx])))

        # Compute the loss
        loss = 0
        for idx in range(num_frames):
            loss += mean_squared_error(X_frames[idx], X_frames_pred[idx])
        print(loss.item() / num_frames)

        # Optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Check for convergence
        if loss < tolerance:
            break

    # Stop computing the gradients for the predictions
    xi_translation = xi_translation.detach()
    xi_rotation = xi_rotation.detach()
    X = X.detach()

    # Convert the predicted transformations to matrices
    T_pred = []
    for idx in range(num_frames):
        T_pred.append(inverse_transformation(exponential_map(xi_translation[idx], xi_rotation[idx])))

    # Apply the predicted transformation
    X = euclidean_coordinates(X)
    X_frames_pred = [ X ]
    for idx in range(num_frames):
        X_frames_pred.append(euclidean_coordinates(apply_transformation(X_frames[idx], T_pred[idx])))

    # Compute RMSE between the transformed point clouds
    error = 0
    for idx in range(num_frames):
        error += torch.sqrt(mean_squared_error(X, X_frames_pred[idx + 1])) / num_frames

    return T_pred, X_frames_pred, error

T = getTransforms(5)
T_pred, X, alignment_error = icp_slam(T)
T_error = 0
for idx in range(T.shape[0]):
    T_error += torch.sqrt(torch.mean((T[idx] - T_pred[idx]) ** 2)) / T.shape[0]
print('Alignment Error:', alignment_error)
print('Error in Transformation:', T_error)

# Visualization
vis = o3d.visualization.Visualizer()
vis.create_window()
PC = [ o3d.geometry.PointCloud() for _ in range(len(X)) ]
for idx in range(len(X)):
    PC[idx].points = o3d.utility.Vector3dVector(X[idx].numpy())
for idx in range(len(X)):
    vis.add_geometry(PC[idx])
vis.poll_events()
vis.update_renderer()
vis.run()
vis.destroy_window()

68956.54375
46975.26875
32688.66875
24807.0828125
20336.91875
17610.8421875
15824.3421875
14581.421875
13672.43125
12978.33515625
12428.11875
11977.778125
11599.09921875
11273.3203125
10987.5296875
10732.5359375
10501.615625
10289.72578125
10093.03984375
9908.6046875
9734.13359375
9567.85078125
9408.37578125
9254.6390625
9105.8078125
8961.23984375
8820.446875
8683.04296875
8548.73046875
8417.27421875
8288.48203125
8162.1921875
8038.2703125
7916.59921875
7797.08125
7679.628125
7564.1796875
7450.6828125
7339.11875
7229.48515625
7121.80625
7016.1328125
6912.5375
6811.1109375
6711.9671875
6615.2203125
6521.0
6429.42734375
6340.624609375
6254.7
6171.74609375
6091.83671875
6015.023828125
5941.341796875
5870.794921875
5803.37109375
5739.037109375
5677.740625
5619.4109375
5563.9671875
5511.315625
5461.35390625
5413.971875
5369.05859375
5326.498828125
5286.175
5247.974609375
5211.784375
5177.494140625
5144.996875
5114.1921875
5084.98125
5057.27109375
5030.973828125
5006.006640625
4982.290234375